In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.15
    Uninstalling langchain-core-0.3.15:
      Successfully uninstalled langchain-core-0.3.15


In [ ]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 9.6 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=3d56bb08eb17d340f440b21d2eabfbb962e4ac0afc4d7d993b7ec6b97f2bf3e1
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os

def extract_investment_content(url, output_folder="extracted_data"):
    # Create a folder for output if it does not exist
    os.makedirs(output_folder, exist_ok=True)

    # Check if the URL is a list (multiple URLs for blogs) or a single URL (for a page with tables)
    if isinstance(url, list):
        print("Detected multiple URLs, treating as blog content...")

        # Use UnstructuredURLLoader for multiple blog URLs
        loader = UnstructuredURLLoader(urls=url)
        documents = loader.load()

        # Concatenate and save each document's content into a single text file
        full_text = "\n\n".join([doc.page_content for doc in documents])
        blog_filename = os.path.join(output_folder, "blog_content.txt")

        with open(blog_filename, "w") as f:
            f.write(full_text)

        print(f"Blog content from multiple URLs saved to {blog_filename}")

        return full_text  # Return the blog content text

    else:
        print("Detected single URL, checking for table data...")

        # Use requests and BeautifulSoup for single URL with table data
        response = requests.get(url)
        html_content = response.content

        # Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(html_content, "html.parser")

        # Check for tables
        tables = soup.find_all("table")
        extracted_tables = []

        if tables:
            print("Detected tables, extracting investment data only...")
            for i, table in enumerate(tables):
                table_data = []
                for row in table.find_all("tr"):
                    cells = row.find_all(["td", "th"])
                    table_data.append([cell.get_text(strip=True) for cell in cells])

                # Convert to DataFrame (assuming first row is the header)
                df = pd.DataFrame(table_data[1:], columns=table_data[0])
                extracted_tables.append(df)

                # Save each table to a separate CSV file
                csv_filename = os.path.join(output_folder, f"table_{i+1}.csv")
                df.to_csv(csv_filename, index=False)
                print(f"Saved table {i+1} to {csv_filename}")

        else:
            print("No tables found and single URL detected. Saving as blog content.")

            # Use UnstructuredURLLoader for the single blog URL
            loader = UnstructuredURLLoader(urls=[url])
            documents = loader.load()
            full_text = documents[0].page_content

            # Save the blog content to a text file
            blog_filename = os.path.join(output_folder, "blog_content_single.txt")
            with open(blog_filename, "w") as f:
                f.write(full_text)

            print(f"Blog content saved to {blog_filename}")

        return extracted_tables if tables else full_text  # Return tables or blog content

# Example usage
urls = [
    "https://www.investopedia.com/articles/investing/010416/angel-investing-vs-crowdfunding-how-raise-money-your-startup.asp",
    "https://www.investopedia.com/articles/investing/030615/investing-social-media-startups-read-first.asp",
    "https://www.investopedia.com/articles/investing/011916/5-questions-ask-investing-startup.asp",
    "https://www.investopedia.com/terms/s/startup-capital.asp",
    "https://www.investopedia.com/how-technology-is-changing-financial-advice-4774011",
    "https://www.investopedia.com/terms/e/entrepreneur.asp",
    "https://www.investopedia.com/articles/basics/11/how-to-pick-a-stock.asp",
    "https://www.investopedia.com/articles/pf/12/private-foundation-start-up.asp"
]
content = extract_investment_content(urls)  # Multiple URLs (for blogs)

single_url = "https://www.openvc.app/search"
content = extract_investment_content(single_url)  # Single URL (for a table)

Detected multiple URLs, treating as blog content...
Blog content from multiple URLs saved to extracted_data/blog_content.txt
Detected single URL, checking for table data...
Detected tables, extracting investment data only...
Saved table 1 to extracted_data/table_1.csv


In [ ]:
import pandas as pd
import json

# Load the blog content text file
with open('/content/extracted_data/blog_content (2).txt', 'r', encoding='utf-8') as file:
    blog_content = file.read()

# Load the CSV file as a DataFrame
table_df = pd.read_csv('/content/Investor Details.csv')

# Convert the DataFrame to a list of dictionaries for JSON compatibility
table_data = table_df.to_dict(orient="records")

# Create a combined structure
combined_data = {
    "blog_content": blog_content,
    "table_data": table_data
}

# Save the combined structure to a JSON file
output_file = '/content/combined_Investment_Data/combined_investment_data.json'
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(combined_data, json_file, indent=4)

print(f"Combined data saved to {output_file}")

Combined data saved to /content/combined_Investment_Data/combined_investment_data.json
